Below we are defining a path for data folder of the analysis  analysis

In [1]:
import pandas as pd
import mne_bids
import mne
from pathlib import Path
import re

# Print the MNE version
print(f"MNE version: {mne.__version__}")

MNE version: 1.10.0


In [3]:
# I have raw mff data inside a  folder called "Raw_data" inside the folder "DATI_PATHS"
# The output and input doesnt need to be in th same folder, we will use later the file from path_raw_bids folde
project_path= "/Users/daniel/PhD/Projects/psd-paths/"
path_raw_mff = f"{project_path}data/DATI_PATHS/Raw_data"
path_raw_bids = f"{project_path}/data/raw/BIDS" #Copy the data to this location

Path(path_raw_bids).mkdir(parents=True, exist_ok=True)


In [4]:
def convert2bids(path, sub, root_path, task='rest', test=True, ):
    """
    Use MNE-BIDS to convert the EEG data to BIDS format.
    """
    raw = mne.io.read_raw_egi(path, preload=True )
    raw.set_montage('GSN-HydroCel-257', match_alias={'VREF' :'Cz'})
    #raw.plot_sensors(show_names=True);
    bids_path = mne_bids.BIDSPath(subject=sub,
                                session='01',
                                task= task,
                                datatype='eeg',
                                root=root_path )
    mne_bids.write_raw_bids(raw, bids_path=bids_path,  format='EEGLAB',allow_preload=True,
                            overwrite=True);

# %% Run the conversion in a loop
# List all files in the directory
files = [file for file in Path(path_raw_mff).glob('*.mff')]

#%%
for fname in files:
    # Use regex to find the first number after "PATHS"
    match = re.search(r'(?<=PATHS_)(\d+)|(?<=PATHS_)(\d+)', fname.stem)
    if match:
        sub = match.group(1)
    convert2bids(fname, sub, root_path = path_raw_bids, task='rest')
    print(f"Converted {fname} to BIDS format")
